INDEED job data search:
Part 1:  project preparation. Build the basic url and get the web page, then show the structure of the website.


In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
url = "https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')

In [4]:
title="oracle database"
location = "Philadelphia"
num_entries = 20
url = f"https://www.indeed.com/jobs?q={title}&l={location}&start={num_entries}"
page = requests.get(url)
soup = BeautifulSoup(page.text, 'lxml')
#print(soup.prettify())

Build functions for job data search
<p>1. Define a function to get more data.</p>

In [8]:
entries = range(0, 50, 10) # how many pages we want to retrieve the data, the second number determines how many pages 
def get_data_soup_list(job_title, city, entries):
    soups = []
    for entry in entries:
        url= f"https://www.indeed.com/jobs?q={job_title}&l={city}&start={entry}"
        soup = BeautifulSoup(requests.get(url).text, 'lxml')
        soups.append(soup)
    return soups;

2. Define a function to retrieve data from indeed

In [9]:
def extract_job_title_from_result(soup):
    jobs = []
    for div in soup.find_all(name='div', attrs = {"class":"job_seen_beacon"}):
        tbody = div.find("tbody")  #go into the table body
        trs = tbody.find_all('tr')
        
        for tr in trs:
            td = tr.find('td', {"class":"resultContent"})

            job_title = tr.find('h2').text
            if job_title.startswith('new'):
                
                job_title = re.sub("^new","",job_title)
            
            company_name=tr.find('span', {"class":"companyName"}).text


            company_location = tr.find('div',{"class":"companyLocation"}).text
            


            if tr.find('div',{"class":"attribute_snippet"}):
                salary = tr.find('div',{"class":"attribute_snippet"}).text
            else:
                salary=None
            if tr.find('a', {"class":"turnstileLink companyOverviewLink"}):
                company_link = tr.find('a', {"class":"turnstileLink companyOverviewLink"})['href']
            else:
                company_link=None
            
            jobs.append((job_title, company_name, company_location, salary, company_link))

    return jobs;
extract_job_title_from_result(soup)

[('Human Resources Generalist II',
  'Crozer Health',
  'Springfield, PA 19064',
  'Full-time',
  '/cmp/Crozer-Health'),
 ('Oracle HCM Cloud Senior Associate',
  'PRICE WATERHOUSE COOPERS',
  'Philadelphia, PA 19103 (Logan Square area)+2 locations',
  None,
  '/cmp/Pwc'),
 ('Director - Software Dev',
  'Intellicomm Inc',
  'Wayne, PA',
  '$115,000 - $125,000 a year',
  None),
 ('Clinical RAVE Programmer', 'TCS', 'Philadelphia, PA', None, '/cmp/Tcs'),
 ('Business Analyst',
  'Software Methods, Inc.',
  'Audubon, PA 19403',
  'Contract',
  None),
 ('AWS DBA',
  'Deloitte',
  'Philadelphia, PA 19103 (Rittenhouse area)',
  'Full-time',
  '/cmp/Deloitte'),
 ('Cloud & Infrastructure Technical Consultant',
  'Instem plc',
  'Conshohocken, PA',
  'Full-time',
  None),
 ('Technology Consulting Associate - Summer/Fall 2022',
  'PRICE WATERHOUSE COOPERS',
  'Philadelphia, PA 19103 (Logan Square area)',
  None,
  '/cmp/Pwc'),
 ('Qlik Developer', 'NR Consulting LLC', 'Chesterbrook, PA', 'Contract',

3. Define a function that we can search different location and different job titles 

In [10]:
entries = range(0, 50, 10) # how many pages we want to retrieve the data, the second number determines how many pages 
def get_data_list(job_title, city, entries):
    jobs_list = []
    soups=get_data_soup_list(job_title, city, entries)

    for soup in soups:
        jobs = extract_job_title_from_result(soup)
        jobs_list.extend(jobs)

    return jobs_list

Use the function to find jobs about data scientists in Philadelphia.

In [11]:
jobs_list = get_data_list("data scientist", 'Philadelphia', entries)
jobs_list

[('Sr. Commercial Data Analyst',
  'Antares Pharma',
  'Newtown, PA',
  '$100,000 - $132,873 a year',
  '/cmp/Antares-Pharma'),
 ('Data Scientist',
  'Equity Risk Sciences, llc',
  'Philadelphia, PA•Remote',
  None,
  None),
 ('Investment Management Fintech Data Scientist',
  'Vanguard',
  'Malvern, PA+7 locations',
  None,
  '/cmp/Vanguard'),
 ('Clinical Laboratory Scientist II (Data Review / Quality Cont...',
  'Drugscan / ACM Global Laboratories',
  'Horsham, PA 19044',
  'Up to $25 an hour',
  None),
 ('Data Scientist II',
  'Burlington Stores',
  'Edgewater Park, NJ 08010',
  None,
  '/cmp/Burlington-Stores'),
 ('Data Scientist II',
  'Comcast',
  'Philadelphia, PA 19103+3 locations',
  None,
  '/cmp/Comcast'),
 ('Data Scientist - $200k (Python/R Tableau)',
  'CyberCoders',
  'Fort Washington, PA 19034',
  '$150,000 - $200,000 a year',
  '/cmp/Cybercoders'),
 ('REMOTE Customer Success Data Admin - $100k + 10% (SQL, BI)',
  'CyberCoders',
  'Philadelphia, PA 19140 (Hunting Park are

4. Write a function to store our data into MySQL. (reference: https://pynative.com/python-mysql-insert-data-into-database-table/)

In [94]:
def store_data_into_MySQL(jobs_list):
    import mysql.connector
    try:
        connection = mysql.connector.connect(host='localhost',
                                        database='DSCI511',
                                        user='sammy',
                                        password='dsci511')
        mysql_insert_query = """INSERT INTO Company (job_title,company_name, location, salary, url) VALUES (%s,%s,%s,%s,%s)"""
    
    
        cursor = connection.cursor()
        cursor.executemany(mysql_insert_query, jobs_list)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into MySQL table")
    
    except mysql.connector.Error as error:
        print(error)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [90]:
store_data_into_MySQL(jobs_list)

75 Record inserted successfully into MySQL table
MySQL connection is closed


5. Retrieve data from MySQL

In [97]:
def get_data_from_db(table_name):
    try: 
        connection = mysql.connector.connect(host='localhost',
                                         database='DSCI511',
                                         user='sammy',
                                         password='dsci511')
    
        my_query = f"SELECT * FROM {table_name}"
        cursor=connection.cursor()
        cursor.execute(my_query)
        records = cursor.fetchall()
        for row in records:
            print("ID: ", row[0])
            if row[1]:
                print("Job Title: ", row[1])
            else:
                print("Job title is not available.")
            if row[2]: 
                print("Company Name: ", row[2])
            else:
                print("Comapny Name is not available.")
            if row[3]:
                print("Location: ", row[3])
            else:
                print("Location is not available.")
            if row[4]:
                print('Salary: '+ row[4])
            else:
                print("Salary is not abailable.")
        connection.close()
        cursor.close()
    except mysql.connector.Error as error:
            print(error)
    finally: 
        if connection.is_connected():
            connection.close()
            cursor.close()
            print("MySQL connection is closed")  

In [98]:
get_data_from_db("Company")

ID:  17
Job Title:  Data Scientist - Delta One Strats
Company Name:  Morgan Stanley
Location:  New York, NY+3 locations
Salary is not abailable.
ID:  18
Job Title:  Data Scientist II - Algorithmic Justice Specialist
Company Name:  American Civil Liberties Union
Location:  New York State+2 locations•Remote
Salary: $115,638 a year
ID:  19
Job Title:  Data Scientist
Company Name:  Alldus
Location:  New York, NY+2 locations•Remote
Salary is not abailable.
ID:  20
Job Title:  Sr Data Scientist
Company Name:  The Trade Desk
Location:  New York, NY 10003 (Greenwich Village area)
Salary is not abailable.
ID:  21
Job Title:  Research Scientist-Center for Data Science
Company Name:  New York University
Location:  New York, NY 10012 (Greenwich Village area)+4 locations
Salary is not abailable.
ID:  22
Job Title:  Data Scientist
Company Name:  RELX
Location:  New York, NY+1 location
Salary is not abailable.
ID:  23
Job Title:  Data Scientist I (CNN +)
Company Name:  WarnerMedia
Location:  New York

6. Reading data from MySql using pandas

In [99]:
def get_data_from_db_with_pandas(table_name):
    import mysql.connector

    import pandas as pd

    my_connection = mysql.connector.connect(
         host="localhost",
         user='sammy',
        password='dsci511',
        database='DSCI511'
    )

    df = pd.read_sql(f'SELECT * FROM {table_name}', my_connection)

    my_connection.close()
    return df

In [100]:
df = get_data_from_db_with_pandas('Company')
df

,id,job_title,company_name,location,salary,url
0,17,Data Scientist - Delta One Strats,Morgan Stanley,"New York, NY+3 locations",None,/cmp/Morgan-Stanley
1,18,Data Scientist II - Algorithmic Justice Specia...,American Civil Liberties Union,New York State+2 locations•Remote,"$115,638 a year",/cmp/American-Civil-Liberties-Union
2,19,Data Scientist,Alldus,"New York, NY+2 locations•Remote",None,None
3,20,Sr Data Scientist,The Trade Desk,"New York, NY 10003 (Greenwich Village area)",None,/cmp/The-Trade-Desk
4,21,Research Scientist-Center for Data Science,New York University,"New York, NY 10012 (Greenwich Village area)+4 ...",None,/cmp/New-York-University
...,...,...,...,...,...,...
85,132,"Head of AI, Machine Learning, Data Science pro...",Xen.ai,"Philadelphia, PA•Remote",None,None
86,133,Chief Data Scientist Predictive Analytics,University of Pennsylvania Health System,"Philadelphia, PA 19104 (University City area)",None,/cmp/University-of-Pennsylvania-Health-System
87,134,Machine Learning Engineer - Opportunity for Wo...,VMware,"Philadelphia, PA•Remote",None,/cmp/Vmware
88,135,AI Data Scientist,Resolution Life,"West Chester, PA",None,None


Retrieve data from craigslist using house data as sample:
<p>Part 1: preparation</p>


In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

The first url of our project:

In [13]:
url = "https://philadelphia.craigslist.org/d/for-sale/search/sss"

1. Define a function to get different locations we can use later

In [16]:
def get_locations(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    areas = soup.find('select', id='areaAbb')
    locations = {}
    for l in areas.find_all('option'):
        locations.update({l.text:l.get('value')})
    return locations

In [18]:
get_locations(url)

{'philadelphia': 'philadelphia',
 'albany, NY': 'albany',
 'allentown': 'allentown',
 'altoona': 'altoona',
 'annapolis': 'annapolis',
 'baltimore': 'baltimore',
 'binghamton': 'binghamton',
 'catskills': 'catskills',
 'central NJ': 'cnj',
 'charlottesville': 'charlottesville',
 'cumberland val': 'chambersburg',
 'delaware': 'delaware',
 'eastern CT': 'newlondon',
 'eastern shore': 'easternshore',
 'eastern WV': 'martinsburg',
 'elmira': 'elmira',
 'finger lakes': 'fingerlakes',
 'frederick': 'frederick',
 'fredericksburg': 'fredericksburg',
 'harrisburg': 'harrisburg',
 'harrisonburg': 'harrisonburg',
 'hartford': 'hartford',
 'hudson valley': 'hudsonvalley',
 'ithaca': 'ithaca',
 'jersey shore': 'jerseyshore',
 'lancaster, PA': 'lancaster',
 'long island': 'longisland',
 'new haven': 'newhaven',
 'new york': 'newyork',
 'norfolk': 'norfolk',
 'north jersey': 'newjersey',
 'northwest CT': 'nwct',
 'oneonta': 'oneonta',
 'poconos': 'poconos',
 'reading': 'reading',
 'richmond, VA': 'ri

2. Define a function to retrieve different subjects 

In [21]:
def get_subjects(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    catAbb = soup.find('select', id='catAbb')
    catabbs = {}
    for cat in catAbb.find_all('option'):
        catabbs[cat.text]=cat.get('value')
    return catabbs


In [22]:
get_subjects(url)

{'community': 'ccc',
 'events': 'eee',
 'for sale': 'sss',
 'gigs': 'ggg',
 'housing': 'hhh',
 'jobs': 'jjj',
 'resumes': 'rrr',
 'services': 'bbb'}

3. Define a function to get the categories

In [23]:
def get_categories(city, catabb):
  url = f"https://{city}.craigslist.org/d/for-sale/search/{catabb}"
  
  page = requests.get(url)
  soup = BeautifulSoup(page.text,'html.parser')
  category = soup.find('select', id = 'subcatAbb')
  categories = {}
  for c  in category.find_all('option'):
      categories[c.text] = c.get('value')
  return categories

4. Find the Philadelphia area house categories

In [25]:
categories = get_categories("Philadelphia", "hhh")
categories

{'all': 'hhh',
 'apartments / housing for rent': 'apa',
 'housing swap': 'swp',
 'office & commercial': 'off',
 'parking & storage': 'prk',
 'real estate - by broker': 'reb',
 'real estate - by owner': 'reo',
 'rooms & shares': 'roo',
 'sublets & temporary': 'sub',
 'vacation rentals': 'vac',
 'wanted: apts': 'hou',
 'wanted: real estate': 'rew',
 'wanted: room/share': 'sha',
 'wanted: sublet/temp': 'sbw'}

5. Define a function that we can get data from different locations and differents categories:

In [33]:
def get_housing_info(city, cat):
    url = f"https://{city}.craigslist.org/search/{cat}"
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    table = soup.find('ul', id="search-results")
    lis = table.find_all('li',{"class":"result-row"})
    houses = []
    for li in lis:
        if li.find('time'):
            time = li.find('time').get('datetime')
        else:
            time=None
        if li.find('a', {"class":"result-title hdrlnk"}):
            
            url = li.find('a', {"class":"result-title hdrlnk"})['href']
        else:
            url = None
        if li.find('a', {"class":"result-title hdrlnk"}):
            detail = li.find('a', {"class":"result-title hdrlnk"}).text
        else:
            detail = None
        if li.find('span',{'class':"result-price"}):
            
            price = li.find('span',{'class':"result-price"}).text
        else:
            price = None
        if li.find('span',{"class":"result-hood"}):
            
            location = li.find('span',{"class":"result-hood"}).text
        else:
            location = None
        if li.find('span',{"class":"housing"}):
            beds_fts = li.find('span',{"class":"housing"}).text
        else:
            beds_fts = None
        houses+=[(detail, location, price, beds_fts, url)]
    return houses

In [37]:
houses = get_housing_info('Philadelphia', 'reb')  # All house data from broker

6. Function to process the raw data

In [38]:
def processing_data(houses):
    import re
    new_houses =[]
    for house in houses:
        descript = house[0]
        location = house[1]
        if house[3]:
            bedroom = re.search(r'\d+br', house[3])
            if bedroom:
                bedroom = bedroom.group(0)
            else:
                bedroom='NA'
            room_sqr_ft = re.search(r'\d+ft', house[3])
            if room_sqr_ft:
                room_sqr_ft = re.search(r'\d+ft', house[3]).group(0)[:-2]
            else:
                room_sqr_ft = 'NA'
        if house[2]:
            price = house[2].replace('$',"").replace(',',"")
        else:
            price = 0
    
        new_houses +=[(descript,location, bedroom, room_sqr_ft,price)]   
    return new_houses


In [44]:
new_houses = processing_data(houses)

7. Function to store the data into database

In [48]:
def store_data_into_MySQL(house_list):
    import mysql.connector
    try:
        connection = mysql.connector.connect(host='localhost',
                                        database='DSCI511',
                                        user='sammy',
                                        password='dsci511')
        mysql_insert_query = """INSERT INTO Houses (description,location,bedroom, room_sqr_ft, price) VALUES (%s,%s,%s,%s,%s)"""
    
    
        cursor = connection.cursor()
        cursor.executemany(mysql_insert_query, house_list)
        connection.commit()
        print(cursor.rowcount, "Record inserted successfully into MySQL table")
    
    except mysql.connector.Error as error:
        print(error)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [49]:
store_data_into_MySQL(new_houses)

120 Record inserted successfully into MySQL table
MySQL connection is closed


8. Reading data from MySql using pandas

In [50]:
def get_data_from_db_with_pandas(table_name):
    import mysql.connector

    import pandas as pd

    my_connection = mysql.connector.connect(
         host="localhost",
         user='sammy',
        password='dsci511',
        database='DSCI511'
    )

    df = pd.read_sql(f'SELECT * FROM {table_name}', my_connection)

    my_connection.close()
    return df

In [51]:
get_data_from_db_with_pandas("Houses")

,id,description,location,bedroom,room_sqr_ft,price
0,1,Finest ranch style living with ultra privacy,(Minutes away ),3br,NA,995000
1,2,Prime Princeton Estate,(Princeton ),3br,NA,999999
2,3,EST. AUTO REPAIR on 5.62 Acres close to area b...,(Cape May Court House ),3br,NA,299900
3,4,Vacant Lot Near Wildwood Beaches!...Residentia...,(Rio Grande ),3br,NA,119900
4,5,Great Cash Flow on Turnkey 5 Unit Year Round B...,(Wildwood Crest ),6br,NA,675000
...,...,...,...,...,...,...
115,116,MONTCO Cash Buyer Alert!!,(Norristown ),4br,1530,65000
116,117,Gem In The Rough,(West Oak Lane ),3br,1280,135000
117,118,4 bedroom home in the heart of Royersford,"(Royersford, PA )",4br,2112,0
118,119,Quiet Neighborhood Gem,"(Pottstown, PA )",4br,1081,0
